In [1]:
import torch
print(torch.__version__)


2.5.1+cpu


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.metrics import QuantileLoss
from torch.utils.data import DataLoader

c:\Users\Asus\Desktop\Nifty PredictionbyMacro\torchenv\lib\site-packages\pytorch_forecasting\models\base_model.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
df = pd.read_csv("finaldata.csv")
df.head()

,Date,vixPrice,vixChange%,niftyPrice,niftyChange %,n5day,n10day,n20day,n1day,n60day,...,RbiinterestAnticepation,USInflation Rate (%),USInflationRate%chng,USInflation Rate (%)Anticepation,IndiaInflationRate(%),IndiaInflationRate(%)chng,IndiaInflationRate(%)Anticepation,IndiaBudgetDatesAnticipation,IndiaElectionDatesAnticipation,UsElectionDatesAnticipation
0,2008-06-05,30.32,1.68,4676.95,1.99,4835.3,5025.45,5135.50,4647.00,4864.25,...,0,9.0,4.651163,0,11.8,26.88172,0,0,0,0
1,2008-06-06,30.32,0.00,4627.80,-1.05,4870.1,4946.55,5081.70,4761.20,4921.40,...,0,9.0,4.651163,0,11.8,26.88172,0,1,1,1
2,2008-06-09,32.43,6.96,4500.95,-2.74,4739.6,4875.05,4982.60,4709.65,4771.60,...,0,9.0,4.651163,0,11.8,26.88172,0,2,2,2
3,2008-06-10,30.23,-6.78,4449.80,-1.14,4715.9,4859.80,5012.65,4747.05,4800.40,...,0,9.0,4.651163,0,11.8,26.88172,0,3,3,3
4,2008-06-11,29.64,-1.95,4523.60,1.66,4585.6,4918.35,4957.80,4733.00,4865.90,...,0,9.0,4.651163,0,11.8,26.88172,0,4,4,4


In [5]:
df= df.drop('DayOfWeek', axis = 1)

In [6]:
df= df.drop('month', axis = 1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4016 entries, 0 to 4015
Data columns (total 78 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Date                               4016 non-null   object 
 1   vixPrice                           4016 non-null   float64
 2   vixChange%                         4016 non-null   float64
 3   niftyPrice                         4016 non-null   float64
 4   niftyChange %                      4016 non-null   float64
 5   n5day                              4016 non-null   float64
 6   n10day                             4016 non-null   float64
 7   n20day                             4016 non-null   float64
 8   n1day                              4016 non-null   float64
 9   n60day                             4016 non-null   float64
 10  nc5day                             4016 non-null   float64
 11  nc10day                            4016 non-null   float

In [8]:
# Preprocess the data
df['Date'] = pd.to_datetime(df['Date'])


In [9]:

# Define a time index for the dataset
df['time_idx'] = np.arange(len(df))


In [10]:

# Define a TimeSeriesDataSet
max_encoder_length = 240  # Past window size
max_prediction_length = 20  # Forecast horizon


In [13]:

# Ensure required columns
features = [col for col in df.columns if col not in ['niftyPrice', 'Date']]
target = 'niftyPrice'

training = TimeSeriesDataSet(
    df,
    time_idx='time_idx',
    target=target,
    group_ids=["group"],  # Replace "group" if grouping is not required
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=[],  # Add static categorical features if any
    time_varying_known_reals=["time_idx"] + features,
    time_varying_unknown_reals=[target],
    add_relative_time_idx=True,
    add_target_scales=True,
)


TypeError: __init__() missing 1 required positional argument: 'group_ids'

In [ ]:

# Create dataloaders
train_dataloader = DataLoader(training, batch_size=64, shuffle=True)


In [ ]:

# Define the TFT Model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    loss=QuantileLoss(),
    reduce_on_plateau_patience=4,
)


In [ ]:

# Train the Model
from pytorch_lightning import Trainer

trainer = Trainer(max_epochs=10, gpus=1)  # Use GPUs if available
trainer.fit(tft, train_dataloader)


In [ ]:

# Evaluate and Visualize
actuals = torch.cat([y[0] for x, y in iter(train_dataloader)])
predictions = tft.predict(train_dataloader)


In [ ]:

# Plot predicted vs actual
plt.figure(figsize=(10, 6))
plt.plot(range(len(actuals)), actuals, label="Actual")
plt.plot(range(len(predictions)), predictions, label="Predicted")
plt.legend()
plt.title("Predicted vs Actual Values")
plt.show()
